# [크리스 리처드슨] 마이크로 서비스 패턴

## 1장 - 모놀리식 지옥에서 벗어나라

### 1.1 - 서서히 모놀리식 지옥에 빠져들다
- 확장 어렵고 가면 갈 수록 개판된다.

### 1.2 - 이 책의 대상 독자
- 기초적인 것들은 알아야 뭘 할 수 있다.

### 1.3 - 이 책의 학습 내용
- MSA를 다룰 수 있게 될 것이다.

### 1.4 - MSA가 답이다
- 확장 큐브 개념
    - 3차원의 큐브를 생각해보자.
    - X 좌표는 인스턴스 확장
    - Y 좌표는 기능 별 분해
    - Z 좌표는 데이터 파티셔닝
    - 이걸 가능하게 해준다.

### 1.5 - MSA의 장단점
- 장점
    - 기능 추가 쉽다.
    - 배포 쉽다.
    - 결함 격리가 쉽다.
    - 신기술 도입이 쉽다.
- 단점
    - 어렵다.

### 1.6 - MSA 패턴 언어
- ...

### 1.7 - 그 너머: 프로세스와 조직
- MSA와 애자일은 찰떡궁합이다.
- CI/CD와도 찰떡궁합이다.

## 2장 - 분해 전략

### 2.1 - MSA란?
- 계층형 아키텍처에서 헥사고날 아키텍처로 넘어가자.
- 도넛과 같은 구조이다.
    - 작은 원은 비즈니스 로직을 가지고 있다.
        - 내부에서 외부로 향하는 연결고리를 '포트'라고 한다.
    - 둘러싼 원은 해당 비스지스를 사용하기 위해 부착된 어뎁터의 영역이다.
        - 외부에서 내부로 향하는 연결고리를 '어뎁터'라고 한다.

### 2.2 - MSA 정의
- 대충 설계하는 단계가 있다.
    - 뭘 해야 하나?
    - 이것들이 필요하겠구만!
    - 그렇다면 하나하나 명세하자.
- 그런데 이 분야는 DDD의 바운디드 컨텍스트라서 DDD를 참고하자.

## 3장 - IPC

### 3.1 - MSA와 IPC
- 메세지는 어떤 포멧으로 전송되나?
    - gRPC, Json, XML, ...
    - 난 Json이 좋다!
- 상호작용은 다음과 같다.
    - o2o인가? o2m인가?
    - 동기적인가? 비동기적인가?

### 3.2 - Sync RPI
- 구조
    - o2o, 동기 == REST API, gRPC API

### 3.3 - Async Messaging
- 구조
    - o2o, 비동기 == 커멘드 메세지 아키텍처
    - o2m, 비동기 == 이벤트 메세지 아키텍처
- 메세지 브로커
    - RabbitMQ, Kafka, ...
    - 메세지를 '적어도 한 번' 전달하는데에 사용됨
    - 그렇기 때문에 메세지의 처리는 '한번만' 되던지 '여러번 처리해도 똑같'던지(멱등) 해야 한다.
        - 멱등? 쉽지 않음, 불가능한 경우도 많음
        - 한번만? 처리하고 저장할 때 '처리했음'을 한 트랜잭션 내에서 기록하면 됨
- 메세지의 발행과 처리를 신뢰할 수 있나?
    - 아웃박스 패턴
        - 대충이지만, 애그리게잇 변경과 동시에 이벤트의 발행을 보장하기 위해서 에그리게잇 변경과 이벤트 발행을 한 트랜잭션 내에서 처리하는 방법
        - 데이터 변경 후 이벤트 보낸다면, 중간에 뻑날 때 이벤트가 유실되니까.
    - 인박스 패턴
        - 메세지를 소비할 때 데이터를 변경 후 '이 이벤트 소비함'을 같이 기록하는 패턴이다.
        - 메세지를 두 번 처리할 수 있으니까.
- 메세지 릴레이 - 메세지 발행 우짤껴?
    - 트랜잭션 로그 테일링 - 딥하게 들어가서 DB의 트랜잭션과 연동된 동작 구현하던가
    - 폴링 발행기 - Outbox 테이블 조회해서 처리 안된 레코드 전송하던가
- 이벤추에이트라는 라이브러리 만들어놨다는데 자바꺼라서 ㅈㅈ

### 3.4 - Async Messaging과 가용성
- 동기적으로 서비스가 얽혀있으면 하나 뻑날때 나머지도 뻑난다.
- 그러니 비동기적으로 구현하자.

## 4장 - 트랜잭션 관리: 사가

### 4.1 - MSA에서 트랜잭션 관리
- 기존 Monolithic 아키텍처는 그냥 하나의 트랜잭션으로 처리 가능한데 MSA서는 거진 불가능.
- 2단계 커밋(XA)는 거의 안쓰인다. 불가능한 경우가 너무 많다.
- 그래서 사가가 뭔데?
    - 트랜잭션 없이 데이터 일관성을 유지하는 메커니즘
    - 간단하게 예를 들자면 이렇다.
        - [] == 대충 트랜잭션같은 녀석
        - N + [1 - 2 + 3 - 4 + 5] => N + 3
        - 모놀리식이라면 [1 - 2 + 3 - 4 + 5]
        - MSA라면 \[1] \[- 2] \[+ 3] \[- 4] \[+ 5]
        - 그러므로 중간에 실패했을 경우 역으로 되돌아오는 과정(+1이라면 -1, ...)을 구현한다.
        - 각 단계를 서비스마다 구현해서 서로 협동하며 처리하는 것이 '사가' 패턴

### 4.2 - 사가 편성
|서비스  | 트랜잭션 | 보상 트랜잭션| 결제 실패 예제 |
| --- | --- | --- | --- | 
| 1. 주문 | createOrder() | rejectOrder() |order.state가 pending(보류) 상태로 생성  |
| 2. 고객 | verifyConsumerDetails() | | 고객 주문 가능 여부 확인 |
| 3. 주방 | createTicket() | rejectTicket() | ticket도 pending(보류) 상태로 생성 |
| 4. 회계 | authorizeCreditCard() | | 신용카드 승인 거부됨 |
| 5. 주방 |  approveTicket() | | ticket.state가 create_rejected(거부)로 변경 |
| 6. 주문 | approveOrder() | | order.state가 rejected(거부)로 변경 |

- 구조(사가 프로세스를 총 3가지의 단계로 분류)
    - 프로세스를 ㅅ자 형태의 구조로 보자.
    - 오르막길, 꼭지점을 지나기 전까지는 보상 트랜잭션으로 롤백할 수 있다
        - Compensatable transaction, 보상 가능 트랜잭션 (1~3)
        - 2는 읽기 전용이라 보상 트랜잭션이 없어도 됨
    - 정상, 여기를 넘어서면 끝까지 굴러가야 한다
        - Pivot Transaction, 피봇 트랜잭션 (4)
    - 내리막길, 어캐 되던지 내려가기만 하면 된다
        - Retriable Transaction, 재시도 가능 트랜잭션 (5~6)

- 구현 방법
    - 코레오그래피 사가 (알아서 각자 앞에서 발생한 이벤트 받아서 협업하기)
        - 주문 <- 고객 <- 주방 <- 회계 <- 주방 <- 주문 형태
        - 전방 서비스가 발행할 이벤트를 후방 서비스가 '알고 있어야'한다는 문제가 생긴다(종속)
    - 오케스트레이션 사가 (일련의 요청/응답 프로세스를 도맡아 처리하는 녀석을 두기)
        - 유한 상태 머신과 관계가 깊다.
        - 주문 사가 <-> [주문, 고객, 주방, 회계]
        - 코레오그래피 사가보다는 오케스트레이션 사가가 좋다.
        - 또한 이 녀석은 '순서화'만 담당하고 로직은 배제하자.

### 4.3 - 비격리 문제 처리
- 트랜잭션은 ACID를 지켜야 한다!
    - Atomicity: 원자성
        - 단위 전체가 성공하던지 전체가 실패하던지 한다.
    - Consistency: 일관성
        - 트랜잭션 완료 시 데이터 형태의 제약 조건이 반드시 지켜져야 한다.
    - Isolation: 고립성
        - 한 트랜잭션 진행 과정에서 사용되는 데이터는 다른 트랜잭션이 수정할 수 없다.
    - Durability: 지속성
        - 일단 커밋됐으면 데이터는 저장되어 있어야 한다.
- 그래서 MSA에서 어떻게 ACID를 지키는데?
    - A : 서비스가 각자 자신만의 트랜잭션 단위를 처리함으로써
    - C : 서비스 내에서는 로컬 DB로, 서비스간 걸친 제약 조건은 서비스로
    - I : ???
    - D : 로컬 DB로
- N+\[1+2+3+4+5]가 N\[+1]\[+2]\[+3]\[+4]\[+5]가 되어 생기는 문제
    - 소실된 업데이트 : 진행이 겹쳐서, 롤백 과정에 다른 사가가 결과를 덮어쓴다면?
    - 더티 읽기 : 진행이 겹쳐서, \[+3]이 \[+5]로 바뀌었다면?
- 해결 방안 종류
    - 시맨틱 락
        - 보상 가능 트랜잭션이 생성/수정하는 레코드에 플래그를 세팅하는 방안
        - 에그리게잇에 state를 둬서 생성시 Pending, 완료시 Approved, 롤백시 Rejected로 삽입
        - 각 플래그마다 어떻게 처리할지에 대한 로직을 설계해야 해서 좀 복잡하긴 함
    - 교환적 업데이트
        - 순서가 상관 없는 경우
        - 딱히 입금이 먼저 되든 출금이 먼저 되든 상관 없음.
    - 비관적 관점
        - 순서를 바꿔서 더티 읽기 가능성을 줄이는 방안
        - 주문 취소로 잔액을 늘리는 과정은 나머지 단계를 다 처리하고 늘리도록.
    - 값 다시 읽기
        - 레코드 업데이트 전에 다시 확인하는 방안
    - 버전 파일
        - 주문 요청과 취소가 동시에 되었을 때, 결제를 처리하는 쪽에서 결제 취소를 받고 결제를 받으면?
        - 이 때 결제 취소를 기록해놓고 결제를 받았다면 해당 결제를 무시하면 그만.
    - 값에 의한
        - ...? 사가 쓸지 분산 트랜잭션 쓸지 고민 더 해 보셔요~ 라는데 사가 쓸거임.

### 4.4 - 구현
- 대략적인 사가와 서비스들 간의 구조
    - 사가는 "각 서비스의 요청용 큐들"을 알고 있어야 하며 메세지를 각 요청 큐들로 보낸다.
    - 각 서비스들은 "사가의 응답 수신용 메세지 큐"를 알고 있어야 하며 응답 메세지를 해당 큐로 보낸다.
    - 그러면 사가가 알아서 조합해서 처리한다.
- 유비쿼터스 언어를 붙여서 설명하자면...
    - CreateOrder 커멘드가 Order 서비스에 전달되면
    - CreateOrderSaga를 생성하고
    - CreateOrderSaga가 Order, Account, Consumer, Kitchen 서비스들의 진입 이벤트 큐에 이벤트 쏘면서 전송하고 SagaResponse 큐로 응답 받아가며 처리하고
    - OrderCreated나 OrderRejected 이벤트를 Order 서비스에 반환해서 종료
        - 아예 오케스트레이션 사가를 별도의 서비스로 분리해서 생각할 수도 있겠구만.
        - 그러면 너무 복잡한가?
- 어케 구현하냐...?
    - 4.4.1 참고
    - 책에서는 유한 상태 머신에 따라 설계하고 진행되는 방식으로 구현되어 있다.
    - 이걸 어찌 파이썬으로 구현하면 되겠습니까!

## 5장 - 비즈니스 로직 설계

### 5.1 - 비즈니스 로직 구성 패턴
- 주문 서비스의 구조
    - 인바운드 어뎁터(request queue, REST API)에서 들어온 주문 요청은 CreateOrder로 변환되어 진입한다.
    - 비즈니스 로직(안쪽 로직)이 이를 처리한다.
    - 아웃바운드 어뎁터가 새로운 이벤트를 발행(event queue)하거나 영속성을 관리한다.

### 5.2 - DDD 애그리게잇 패턴
- 엔티티, 값객체, 에그리게잇, 도메인 서비스, 도메인 이벤트, 리포지토리...
- 이런 것들은 DDD를 통해 학습하기로 하자.
- 지금은 Order 엔티티와 OrderLineItem을 하나의 에그리게잇으로 관리한다고 생각하자.
- 그러면 Order 서비스는 Order 에그리게잇과 여러 Order Saga 객체들을 지니는 형태가 된다.

### 5.3 - 도메인 이벤트 발행
- 에그리게잇의 생성/변경마다 이벤트를 발행하면 된다.
    - 결과적 일관성 유지에 활용
    - 사가 프로세스 단계에 활용
    - Notification 기능 활용
    - ...
- 도메인 이벤트란?
    - 어떤 일이 발생하였음을 외부에 알리는 메세지 정도로 해석
        - 이에 따라 갱신되어야 하는 여러 프로세스에 필요한 인자들을 담고 있는 형태
    - 일반적으로 명사 + 과거 분사형 동사 형태로 기록됨
        - OrderCreated, OrderRejected, ...
- 이벤트 스토밍
    - 이건 [동영상](https://www.youtube.com/watch?v=QUMERCN3rZs)으로 설명
- 구현
    - 책은 재미있게도 에그리게잇 메서드의 return이 events이도록 구현되어 있다.
    - 에그리게잇에 events 속성을 설정하거나 ContextVar를 활용해서 events를 수집하는 방식도 있는데, 책의 방식이 좀 더 합리적인 것 같기도 하다.
    - 좀 더 생각해보자.

### 5.4 - 주방 서비스 비즈니스 로직
- ...

### 5.5 - 주문 서비스 비즈니스 로직
- ... 5-17 그림을 보니, 이것도 에그리게잇 수정과 이벤트 발행 과정이 분할되어 있는데...? 흠. 뭐지?
- 이게 이벤트 발행을 보장하는 코드인가?
- 아니면 어차피 6장에서 이벤트 소싱으로 해결할 수 있다고 생각해서 제외한건가?

## 6장 - 비즈니스 로직 개발: 이벤트 소싱

### 6.1 - 이벤트 소싱 응용 비즈니스 로직 개발
기존에 에그리게잇의 엔티티, 로우들을 RDBMS의 각 레코드에 매핑하고 불러오는 과정을 통해 영속화를 처리했다. Order는 Order 테이블에, OrderLineItem은 OrderLineItem에 매핑


문제점들
- 그래프 형태면 어떻게 저장할겨? (객체-관계 임피던스 부정합)
- 현재 애그리게잇 상태만을 알 수 있을 뿐 (이력 없음)
- 로깅은 어디에서 처리함? (로깅 로직 위치)
- 이벤트 릴레이도 직접 구현해야 하고, 애그리게잇 생성/변경 시 이벤트가 발생한다는 보장을 어떻게 할건데? (이벤트 발행 로직)

기존 상태 + 발생한 일들... => 지금 상태가 아닐까? 

초기 상태가 0이고 여러 사칙연산 이벤트들이 영속화된다면 우리는 이벤트들을 통해 현재 값을 도출할 수 있잖아?

Events 테이블

| Event ID | Event Type | Entity Type | Entity ID | Event Data |
| --- | --- | --- | --- | --- |
| 102 | OrderCreated | Order | 101 | {...} |
| 103 | OrderApproved | Order | 101 | {...} |
| 104 | OrderShipped | Order | 101 | {...} |
| 105 | OrderDelivered | Order | 101 | {...} |
| 106 | ... | ... | ... | ... |

- aggregate.process(some_command) => List\[Event]
    - 애그리게잇은 커멘드를 받아 이벤트를 생성한다.
- aggregate.apply(event) => None
    - 애그리게잇은 이벤트를 통해 자신의 현재 상태를 반영한다.

```py
class Order(Aggregate):
    def __init__(self, *args, **kwargs):
        ...
    
    def process(command: Command) => List[Event]:
        # ... 여기서는 오버로딩이 없는 파이썬이 약간 불편하네.
        # 타입을 검사하면서 검증/이벤트 추출을 진행하던지
        # 아니면 command 객체에 process와 apply 동작을 위임하는 방식으로 구현해야 겠구만.
        return ...
    
    def apply(event: Event) => None:
        # 이것도 마찬가지.
        ...
```

폴링 방식의 이벤트 발행
- SELECT * FROM EVENTS WHERE EVENT_ID > ? ORDER BY EVENT_ID ASC
    - 문제가 있다. EVENT_ID가 2인 이벤트가 커밋되고 나서 1인 이벤트가 커밋되어 버리면 이벤트가 유실된다.
- SELECT * FROM EVENTS WHERE PUBLISHED = 0 ORDER BY EVENT_ID ASC
    - 이벤트 릴레이가 이벤트를 발행하고 난 뒤에
    - UPDATE EVENTS SET PUBLISHED = 1 WHERE EVENT_ID = ?

트랜잭션 로그 테일링 방식으로 이벤트 발행
- 난 이걸 할 자신이 없다.

스냅샷이란?
- 만약 한 에그리게잇의 event가 한 200개 쯤 된다고 가정해보자. 애그리게잇 만들고 이벤트 200개 가져오고 for문으로 process 돌리고 하는 비용이 어마어마하다. 
- 190개의 이벤트가 적용된 에그리게잇의 상태를 '스냅샷'으로 저장하고, 우리는 해당 스냅샷 + 10개의 이벤트를 활용해 현재 에그리게잇을 가져올 수 있게 된다.
- JSON 형태로 스냅샷을 떠서 저장하고, 애그리거트 생성자는 해당 스냅샷으로 인스턴스를 초기화한다.

EVENTS TABLE
| event_id | event_type | entity_type | entity_id | event_data | published |
| --- | --- | --- | --- | --- | --- |
| 103 | SomeEvented | SomeAggregate | 101 | {...} | 1 |

SNAPSHOTS TABLE
| event_id | entity_type | event_id | snapshot_data |
| --- | --- | --- | --- |
| 103 | SomeAggregate | 101 | some_json_data |

메세지 컨슈머의 입장에서 중복 메세지를 제거하는 방법
- RDBMS일 경우
    - PROCESSED_MESSAGES 테이블과 EVENTS 테이블 삽입 트랜잭션을 동시에 수행하기
- NoSQL의 경우
    - 핸들러 동작이 무조건 이벤트를 발행하도록 하고, EVENTS 테이블에 published가 True인 이벤트를 기록하는 방법

버저닝
- 하위 호환 가능
    - 새 애그리게잇 정의
    - 새 이벤트 타입 추가
    - 새 필드 추가
- 하위 호환 불가능
    - 기존 애그리게잇 삭제
    - 애그리게잇 이름 변경
    - 이벤트 타입 삭제
    - 이벤트 타입명 변경
    - 필드 삭제
    - 필드명 변경
    - 필드 타입 변경

하지만, 만약 SomeEvent(v2)와 SomeEventV1이 있을 경우 우리는 SomeEventV1을 SomeEvent로 가공하는 과정만 거칠 수 있다면(업캐스터) 호환되도록 구성할 수 있다.

이벤트 소싱의 장단점
- 장점
    - 도메인 이벤트의 발행을 보장
    - 애그리게잇 변경 이력 보존
    - 객체 관계형 매핑 불일치 문제를 대부분 방지
    - 개발자에게 타임머신을 제공
- 단점
    - 이벤트 소싱과 메세징에 대한 학습 곡선
    - 이벤트 스키마 변경시의 처리
    - 데이터 삭제 및 쿼리의 난이도
        - 일반적으로 소프트 삭제로 처리함. 안된다? 그러면 암호화하고 해당 키를 제거해버림.
        - 쿼리가 어려우니 일반적으로 CQRS로 해결함.

### 6.2 - 이벤트 저장소(이벤트 스토어) 구현

이미 존재하는 여러 이벤트 저장소 프로그램이 있다. Event Store, Lagom, Axon, Eventuate, ... 직접 구현해서 쓸거면 그리 하고 기존 서비스 쓰려면 그리 하라.

구조
- 이벤트 데이터베이스 (DB)
    - EVENTS : event_id, event_type, event_data, entity_type, entity_id, triggering_event
        - pk = event_id (auto incremented?)
    - ENTITIES : entity_type, entity_id, entity_version, ...
        - pk = entity_type, entity_id
    - SNAPSHOTS : entity_type, entity_id, entity_version, snapshot_type, snapshot_json, triggering_events, ...
        - pk = entity_type, entity_id, entity_version
- 이벤트 릴레이 (폴링, 트랜잭션 로그 테일러)
- 이벤트 브로커 (RabbitMQ, Kafka, ...)

대략적인 구현 방식은 책에 나와있지만, MSA With Python과 개념은 유사한 듯 보인다. 추후 다시 돌아와서 살펴보자.

### 6.3 - 사가와 이벤트 소싱의 결합

이벤트 소싱에서 사가와 트랜잭션
- RDBMS에서는 이벤트 스토어 업데이트 및 사가 오케이스테이션 과정을 한 트랜잭션으로 묶을 수 있다.
    - OrderCreated로 뚝딱
- NoSQL은 그게 안되니까, 사가가 처리하는 과정을 별도의 이벤트로 정의해서 처리하면 된다.
    - OrderCreated가 들어오면 CreateOrderSaga가 실행됨

이벤트 소싱에서 사가와 이벤트 참여 서비스
- RDBMS는 한 트랜잭션으로 묶을 수 있으니 별 문제 안될 것이다.
- NoSQL에서는 두 가지 문제가 있다.
    - 커맨드 메세지 멱등하게 처리
        - 메세지 받아서 처리할 때 들어오는 커맨드 메세지의 ID를 기록하여 중복 제거
    - 응답 메세지 원자적으로 전송
        - ...
        - 솔직히, 읽어도 이해가 안된다. 아직 준비가 안되었다고 판단하고 넘긴다.

## 7장 - MSA 쿼리 구현

### 7.1 - API 조합 패턴 응용 쿼리
Order에 대한 처리를 여기저기서 수행했다고 한다면 Order에 대한 정보를 어떻게 가져올 것인가? 주문, 주방, 배달, 회계 모든 API를 다 조회해야하나?

방법은 검색 서비스를 전방에 두고 나머지 서비스들을 뒤에서 두어, 요청을 중개하는 녀석을 두는 방식이 있다.

### 7.2 - CQRS 패턴
그러나 몇가지 문제가 있다.
- API 조합기를 따로 두는 패턴은 비효율적인 인-메모리 조인을 수행해야 한다.
- 쿼리하기 비효율적인 데이터 구조일 때가 많다.

CQRS란, 커멘드와 쿼리를 구분지어 사용해야 한다는 뜻이다. 에그리게잇 영속 저장소와 쿼리용 저장소를 따로 두어 활용할 때의 장점이 있다. 
- MSA에서 쿼리를 효율적으로 구현할 수 있다.
- 이벤트 소싱은 쿼리가 거의 불가능하기 때문에 쿼리 기능을 지원해줄 수 있다.
- 쿼리를 효율적으로 구현할 수 있다.

대충 최종 일관성이니 뭐니 하는 느낌이다.

물론 단점도 있다.
- 복잡하다.
- 데이터 갱신의 시차가 존재한다.

### 7.3 - CQRS 뷰 설계
대표적으로 두 개의 어뎁터가 있다. 하나의 어뎁터는 이벤트를 받아 데이터를 갱신하는 어뎁터(이벤트 핸들러)이고, 나머지 하나는 쿼리를 통해 DB 데이터를 조회하는 어뎁터이다.

설계 결정 요소들
- DB 선정, 스키마 설계
    - JSON을 PK로 조회한다면 MongoDB나 Key-Value DB 사용
    - JSON을 쿼리해야 한다면 MongoDB
    - 텍스트 기반의 쿼리가 필요하다면 Elastic Search
    - 그래프를 쿼리해야 한다면 Neo4j
    - 고전적인 방식 SQL DB
- DAO 설계 시 멱등성/동시성 등 다양한 문제를 고려
- 새로운 뷰가 설계되거나 스키마를 변경할 시 뷰를 재빌드할 수 있는 수단을 마련


### 7.4 - CQRS 뷰 구현: DynamoDB 응용
설계 시 검토점
- 테이블 설계
- 쿼리용 인덱스 정의 및 구현
- 쿼리 결과 페이지네이션
- 업데이트
- 중복 이벤트 감지

구현시 DB마다 구현하는 방식이 상이할 것이다.

## 8장 - 외부 API 패턴

### 8.1 - 외부 API 설계 이슈
클라이언트가 API 서비스들을 직접적으로 호출하는 설계의 단점
- 서비스가 여러개이면 여러 번 요청해야 하고 만약 클라이언트가 동기로 구현되어 있다면 버벅거릴 것.
    - 모놀리식일 때에는 괜찮았는데.
- 클라이언트와 서비스가 강결합되어 있어 변경에 있어 자유롭지 못함.
- 방화벽으로 인해 효율이 떨어질 수 있음.

### 8.2 - API 게이트웨이 패턴
외부 클라이언트가 API에 요청을 하는 단일 창구 역할의 서비스를 의미. 방화벽 내부에 있어서 외부 클라이언트의 요청을 받아 서비스들로 라우팅하고(필요하다면 프로토콜을 변환) 응답을 조합하여 반환하는 역할을 담당.

기존에 제공되는 서비스를 사용하던지 직접 구현하던지 할 수 있다. 장단점은 당연하니 생략.

추가적으로 엣지 기능도 맡아 처리할 수 있음.
- 인증/인가
- 쓰로틀
- 캐싱, 지표 수집
- 로깅

그래서 누가 개발하는가? 각 클라이언트마다 API 게이트웨이를 따로 두자. BFF 패턴에 대해 검색해보자.

장단점
- 내부 구조의 캡슐화 및 API 최적화
- 추가 과정

고려사항
- 성능, 확장성
- 비동기
- 부분 실패 처리
- 애플리케이션 아키텍처에서 선량한 시민 되기(??? 이게 뭐지?)

### 8.3 - API 게이트웨이 구현
자바 코드라서 넘김. GraphQL도 유용할 수 있으니 확인해봐~는 참고.

# 9장부터